# Server Code(chatting room)

In [ ]:
import struct
import socket
import threading
from datetime import datetime



class ChatServer:

    def __init__(self, host, port=12345):
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.bind((host, port))
        self.server_socket.listen(5)
        self.clients = []
        self.client_list = []
        self.chatHistory = []

    def broadcast(self, message):
        # with threading.Lock():
        message = self.sendPreprocess(message)
        for client in self.clients:
            try:
                client.send(message)
            except Exception as e:
                print(f"Error sending message: {e}")
                self.clients.remove(client)
                self.client_list.remove(client)
    
    def sendPreprocess(self, item):
        if isinstance(item, str):
            item_bytes = item.encode('utf-8')
            item_length = struct.pack('!I', len(item_bytes))
            item = item_length + item_bytes
            return item
        elif isinstance(item, bytes):
            item_length = struct.pack('!I', len(item))
            item = item_length + item
            return item
        else:
            raise TypeError("item must be str or bytes")

    def handle_client(self, client_socket, address):
        print(f"New connection: {address}")
        self.clients.append(client_socket)
        self.client_list.append(client_socket)
        print(self.client_list)
        client_socket.send(self.sendPreprocess(f"Console: 输入你的名字\n输入msg [名字] [消息]可以私发一条消息"))
        name = self.receivePreprocess(client_socket).decode('utf-8')

        if name:
            self.client_list[self.clients.index(client_socket)] = name
            print(self.client_list)
            self.broadcast(f'{name}上线了')
            
            self.tempClientList = []
            for each in self.client_list:
                if not isinstance(each, str):
                    continue
                self.tempClientList.append(each)
            self.broadcast(f'在线用户: {self.tempClientList}')
        try:
            while True:
                message = self.receivePreprocess(client_socket).decode('utf-8')
                datetimeStr = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                # message = message.decode('utf-8')
                if 'msg' in message:
                    if len(message.split(' ')) < 3:
                        client_socket.send(self.sendPreprocess(f'Console: 格式错误\n私发一条消息: msg [名字] [消息]'))
                        continue
                    clientTalkTo = message.split(' ')[1]
                    if clientTalkTo not in self.client_list:
                        client_socket.send(self.sendPreprocess(f'Console: {clientTalkTo}不存在'))
                        continue
                    message = message.split(' ')[2]
                    messageTo = f'{datetimeStr} {name}对你说: {message}'
                    messageSelf = f'{datetimeStr} 你对{clientTalkTo}说: {message}'
                    self.clients[self.client_list.index(clientTalkTo)].send(self.sendPreprocess(messageTo))
                    client_socket.send(self.sendPreprocess(messageSelf))
                    self.chatHistory.append(message)
                    continue
                message = f'{datetimeStr} {name}: {message}'
                self.chatHistory.append(message)
                if not message:
                    break
                print(f"Received message: {message}")
                self.broadcast(message)
        finally:
            print(f"Connection closed: {address}")
            self.clients.remove(client_socket)
            self.client_list.remove(name)
            client_socket.close()
            self.broadcast(f'{name}下线了')
            self.broadcast(f'在线用户: {self.client_list}')
            # if self.clients == []:
            #     print("No clients connected.")
            #     self.close()

    def start(self):
        print("Server is listening...")
        while True:
            client_socket, address = self.server_socket.accept()
            
            threading.Thread(target=self.handle_client, args=(client_socket, address)).start()
    
    def receivePreprocess(self, client_socket):

        header = client_socket.recv(4)

        if not header:
            return -1
        
        item_length = struct.unpack('!I', header)[0]
        item = client_socket.recv(item_length)
        return item
    
    def close(self):
        self.server_socket.close()
        for client in self.clients:
            client.close()
        print("Server closed.")
        for each in self.chatHistory:
            with open(r"C:\Users\test\Desktop\data_collection_code_v3\Dices_Street\chatHistory.txt", "a") as f:
                f.write(each + "\n")
        print("Chat history saved.")

if __name__ == "__main__":
    host = socket.gethostname()
    ip = socket.gethostbyname(host)
    server = ChatServer(ip)
    print(ip)
    server.start()

192.168.3.56
Server is listening...
New connection: ('192.168.3.56', 57838)
[<socket.socket fd=2252, family=2, type=1, proto=0, laddr=('192.168.3.56', 12345), raddr=('192.168.3.56', 57838)>]
['1']
Received message: 2025-01-02 18:19:51 1: 1
Received message: 2025-01-02 18:19:52 1: 1


Exception in thread Thread-5 (handle_client):
Traceback (most recent call last):
  File "d:\Anaconda\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "d:\Anaconda\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\27818\AppData\Local\Temp\ipykernel_28764\2861706300.py", line 63, in handle_client
  File "C:\Users\27818\AppData\Local\Temp\ipykernel_28764\2861706300.py", line 107, in receivePreprocess
ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接。


Connection closed: ('192.168.3.56', 57838)


In [ ]:
import socket
host = socket.gethostname()
ip = socket.gethostbyname(host)
ip